# Create the similarity matrix
In 3 simple steps:

Create the big users-items table

Replace NaNs with zeros

Compute pairwise cosine similarities

1. Create the big users-items table.
We are just reshaping (pivoting) the data, so that we have users as rows and restaurants as columns. We need the data to be in this shape to compute similarities between users in the next step.

In [21]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

In [22]:
df_links = pd.read_csv(r'links.csv')
df_movies = pd.read_csv(r'movies.csv')
df_ratings = pd.read_csv(r'ratings.csv')
df_tags = pd.read_csv(r'tags.csv')

In [23]:
df_ratings


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [24]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [25]:
X = pd.pivot_table(data=df_ratings, 
                                 values='rating', 
                                 index='userId', 
                                 columns='movieId')

In [26]:

X.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Replace NaNs with Nearest Neighbor Imputation ( KNNImputer)
The cosine similarity can't be computed with NaN's

In [27]:
imputer = KNNImputer()
# fit on the dataset
imputer.fit(X)
# transform the dataset
users_items = imputer.transform(X)

In [8]:
users_items=pd.DataFrame(users_items)

In [10]:
users_items

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,4.0,3.6,4.0,2.8,3.4,4.0,3.2,3.0,2.9,3.6,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
1,4.0,3.6,2.9,2.1,2.2,3.7,2.8,3.0,2.4,3.3,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,3.7,3.0,2.1,2.7,2.8,3.3,2.5,2.8,2.8,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.9,3.6,3.6,2.5,3.2,3.2,3.6,3.0,3.4,3.7,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,4.0,3.4,3.3,2.1,2.6,4.0,3.2,2.6,3.5,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.5,3.6,3.3,2.7,2.8,3.8,2.5,2.4,2.6,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
606,4.0,3.8,4.0,2.8,3.6,3.8,4.0,2.6,3.3,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,2.5,2.0,2.0,2.1,2.9,4.3,3.2,2.8,2.6,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
608,3.0,3.3,3.5,2.5,3.1,3.9,3.3,2.8,2.5,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


### 3. Compute cosine similarities

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)
user_similarities.head()

,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
0,1.000000,0.996213,0.992449,0.993509,0.995572,0.996142,0.995128,0.996736,0.997108,0.994216,...,0.997517,0.996158,0.991202,0.996084,0.995466,0.994857,0.996362,0.992646,0.994804,0.996080
1,0.996213,1.000000,0.995217,0.995293,0.996924,0.996247,0.996134,0.997006,0.996965,0.995733,...,0.996930,0.996470,0.993640,0.996949,0.996762,0.996476,0.996319,0.994788,0.996969,0.995925
2,0.992449,0.995217,1.000000,0.993965,0.995390,0.993638,0.994232,0.994433,0.994265,0.994317,...,0.993661,0.994418,0.992357,0.994687,0.995655,0.993691,0.993524,0.993295,0.995957,0.993480
3,0.993509,0.995293,0.993965,1.000000,0.995789,0.994336,0.995825,0.995550,0.995120,0.995182,...,0.994394,0.995280,0.992583,0.995572,0.995618,0.994768,0.994703,0.991824,0.995818,0.993545
4,0.995572,0.996924,0.995390,0.995789,1.000000,0.995589,0.996274,0.997073,0.996720,0.995515,...,0.996410,0.996715,0.993842,0.996921,0.996517,0.996318,0.996158,0.994206,0.997524,0.995359


### Building the recommender step by step:
Let's focus on one random user (user 1) and compute the recommendations only for this user, as an example. Then, we will build a function that can compute recommendations for any users. We will follow these steps:

Compute the weights.

Find movies user 1 has not rated.

Compute the ratings user 1 would give to those unrated movies.

Find the top 5 movies from the rating predictions.

### 1. Compute the weights
Here we will exclude user 1 using .query().

In [20]:
user_Id = 5
weights = (
    user_similarities.query("userId!=@user_Id")[user_Id] / sum(user_similarities.query("userId!=@user_Id")[user_Id])
          )


UndefinedVariableError: name 'userId' is not defined

### 2. Find restaurants user 1 has not rated.
We will exclude our user, since we don't want to include them on the weights.

In [52]:
users_items.loc[userId,:]==0

movieId
1         False
2          True
3         False
4          True
5          True
          ...  
193581     True
193583     True
193585     True
193587     True
193609     True
Name: 1, Length: 9724, dtype: bool

In [53]:
# select restaurants that the inputed user has not visited
not_visited_movies = users_items.loc[users_items.index!=userId, users_items.loc[userId,:]==0]
not_visited_movies.T

userId,2,3,4,5,6,7,8,9,10,11,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3. Compute the ratings user 1 would give to those unrated restaurants.

In [19]:
not_visited_movies.T.dot(weights)

movieId
2         0.842127
4         0.027652
5         0.275115
7         0.321403
8         0.046876
            ...   
193581    0.000286
193583    0.000250
193585    0.000250
193587    0.000250
193609    0.002973
Length: 9492, dtype: float64

In [20]:
# dot product between the not-visited-restaurants and the weights
weighted_averages = pd.DataFrame(not_visited_movies.T.dot(weights), columns=["predicted_rating"])
weighted_averages

,predicted_rating
movieId,
2,0.842127
4,0.027652
5,0.275115
7,0.321403
8,0.046876
...,...
193581,0.000286
193583,0.000250
193585,0.000250


### 4. Find the top 5 movies from the rating predictions

In [21]:
recommendations = weighted_averages.merge(df_movies, left_index=True, right_on="movieId")
recommendations.sort_values("predicted_rating", ascending=False).head()

,predicted_rating,movieId,title,genres
277,2.654727,318,"Shawshank Redemption, The (1994)",Crime|Drama
507,2.087327,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
659,1.859548,858,"Godfather, The (1972)",Crime|Drama
2078,1.663564,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery
3638,1.624820,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy


### Function:
Make a function that recommends the top n movies to an inputted userId

In [45]:
def user_movie_similarity(userId=1,n=10,user_movie=users_items,movie_names=df_movies):
  userId=int(input("What is your userId "))
  n=int(input("How many movies do you want to get "))
  user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)
  weights = (
    user_similarities.query("userId!=@userId")[userId] / sum(user_similarities.query("userId!=@userId")[userId])
          )
  not_visited_movies = users_items.loc[users_items.index!=userId, users_items.loc[userId,:]==0]
  weighted_averages = pd.DataFrame(not_visited_movies.T.dot(weights), columns=["predicted_rating"])
  recommendations = weighted_averages.merge(df_movies, left_index=True, right_on="movieId")
  return recommendations.sort_values("predicted_rating", ascending=False).head(n)
  

In [56]:
user_movie_similarity()

What is your userId 200
How many movies do you want to get 10


,predicted_rating,movieId,title,genres
510,2.241383,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
507,1.878633,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
461,1.823776,527,Schindler's List (1993),Drama|War
46,1.800264,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
1503,1.765395,2028,Saving Private Ryan (1998),Action|Drama|War
659,1.731600,858,"Godfather, The (1972)",Crime|Drama
2078,1.652478,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery
322,1.535623,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX
520,1.516728,608,Fargo (1996),Comedy|Crime|Drama|Thriller
398,1.481389,457,"Fugitive, The (1993)",Thriller
